In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
import torch


In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Используемое устройство: {device}")

Используемое устройство: cuda


In [3]:
image = cv2.imread(r"C:\Users\Alex\Desktop\2.png")

In [4]:
# Рисуем прямоугольник
points = np.array([
    [0, 230],  # верхний левый
    
    [800, 170],  # верхний правый
    [800, 670],  # нижний правый
    [0, 670]   # нижний левый
], dtype=np.int32)

# Рисуем замкнутый многоугольник (прямоугольник)
cv2.polylines(
    image,
    [points],          # массив точек
    isClosed=True,     # замкнуть контур
    color=(0, 255, 0), # цвет
    thickness=2,
    lineType=cv2.LINE_AA
)

cv2.imshow('Polylines Rectangle', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:

# Загрузка модели YOLO
model = YOLO('yolov8n.pt').to(device)




In [9]:
# Видео
cap = cv2.VideoCapture("H:\stam_CV\cvtest.avi")  # Замените на путь к вашему видео
# Заданный многоугольник (например, зона перед шлагбаумом)
polygon = np.array([
    [0, 483 ],    # верхний левый
    [1680, 460],  # верхний правый
    [1680, 1400],  # нижний правый
    [0, 1400]     # нижний левый
], dtype=np.int32)


def box_intersects_polygon(box, polygon):
    """Проверка, входит ли хотя бы угол bbox внутрь полигона."""
    x1, y1, x2, y2 = map(int, box)
    box_points = [
        (x1, y1), (x2, y1),
        (x2, y2), (x1, y2)
    ]
    for point in box_points:
        if cv2.pointPolygonTest(polygon, point, False) >= 0:
            return True
    return False

def draw_dashed_line(img, pt1, pt2, color, thickness=1, gap=10):
    """Рисование пунктирной линии между двумя точками."""
    dist = ((pt1[0] - pt2[0])**2 + (pt1[1] - pt2[1])**2)**0.5
    dash_count = int(dist // gap)
    for i in range(dash_count):
        start_x = int(pt1[0] + (pt2[0] - pt1[0]) * (i / dash_count))
        start_y = int(pt1[1] + (pt2[1] - pt1[1]) * (i / dash_count))
        end_x = int(pt1[0] + (pt2[0] - pt1[0]) * ((i + 0.5) / dash_count))
        end_y = int(pt1[1] + (pt2[1] - pt1[1]) * ((i + 0.5) / dash_count))
        cv2.line(img, (start_x, start_y), (end_x, end_y), color, thickness)

def draw_dashed_polygon(img, polygon, color=(0, 255, 0), thickness=2, gap=15):
    """Рисует пунктирный многоугольник."""
    for i in range(len(polygon)):
        pt1 = tuple(polygon[i])
        pt2 = tuple(polygon[(i + 1) % len(polygon)])
        draw_dashed_line(img, pt1, pt2, color, thickness, gap)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
        # Подготовка кадра для модели
    if device == 'cuda':
        # Конвертация в тензор и изменение порядка каналов
        frame_tensor = torch.from_numpy(frame).float().to(device)
        frame_tensor = frame_tensor.permute(2, 0, 1)  # HWC -> CHW
        frame_tensor = frame_tensor.unsqueeze(0)      # CHW -> BCHW (добавляем batch dimension)
        frame_tensor /= 255.0                         # Нормализация [0,255] -> [0,1]
        
        # Проверка кратности 32 (требование YOLO)
        h, w = frame_tensor.shape[2], frame_tensor.shape[3]
        if h % 32 != 0 or w % 32 != 0:
            new_h = (h // 32) * 32
            new_w = (w // 32) * 32
            frame_tensor = torch.nn.functional.interpolate(
                frame_tensor, size=(new_h, new_w), mode='bilinear', align_corners=False
            )
        
        results = model(frame_tensor)[0]
    else:
        results = model(frame)[0]

    

    output_frame = frame.copy()
    draw_dashed_polygon(output_frame, polygon)

    boxes = []
    scores = []

    for box in results.boxes.data.tolist():
        x1, y1, x2, y2, score, cls = box
        class_id = int(cls)
        class_name = model.names[class_id]

        if class_name in ['car', 'truck', 'motorcycle', 'bus']:
            if box_intersects_polygon((x1, y1, x2, y2), polygon):
                boxes.append([x1, y1, x2, y2])
                scores.append(score)

    # Применяем Non-Maximum Suppression
    indices = cv2.dnn.NMSBoxes(
        bboxes=[[int(x1), int(y1), int(x2 - x1), int(y2 - y1)] for x1, y1, x2, y2 in boxes],
        scores=scores,
        score_threshold=0.3,
        nms_threshold=0.4
    )

    vehicle_count = 0
    for i in indices:
        i = i[0] if isinstance(i, (tuple, list, np.ndarray)) else i
        x1, y1, x2, y2 = map(int, boxes[i])
        vehicle_count += 1

        cv2.rectangle(output_frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
        cv2.putText(output_frame, "vehicle", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    # Формируем правильный текст в зависимости от количества машин
    if vehicle_count == 0:
        status_text = "nothing detected"
    elif vehicle_count == 1:
        status_text = "1 vehicle in the zone of entry"
    else:
        status_text = f"{vehicle_count} vehicles in the zone of entry"
    
    print(status_text)
    cv2.putText(output_frame, status_text, (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.imshow("Detection", output_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Alex\AppData\Local\Temp\ipykernel_5344\2654534018.py:2: SyntaxWarning: invalid escape sequence '\s'
  cap = cv2.VideoCapture("H:\stam_CV\cvtest.avi")  # Замените на путь к вашему видео



0: 1504x2688 1 person, 1 car, 1 bus, 2 trucks, 1 bottle, 1 cake, 1 refrigerator, 1 clock, 430.6ms
Speed: 19.1ms preprocess, 430.6ms inference, 1153.9ms postprocess per image at shape (1, 3, 1504, 2688)
nothing detected

0: 1504x2688 1 person, 1 car, 1 bus, 2 trucks, 1 bottle, 1 cake, 28.3ms
Speed: 0.0ms preprocess, 28.3ms inference, 12.6ms postprocess per image at shape (1, 3, 1504, 2688)
nothing detected

0: 1504x2688 1 car, 1 bus, 3 trucks, 1 bottle, 1 cake, 25.3ms
Speed: 0.0ms preprocess, 25.3ms inference, 7.7ms postprocess per image at shape (1, 3, 1504, 2688)
nothing detected

0: 1504x2688 1 car, 1 bus, 3 trucks, 1 bottle, 1 cake, 1 refrigerator, 25.0ms
Speed: 0.0ms preprocess, 25.0ms inference, 5.5ms postprocess per image at shape (1, 3, 1504, 2688)
nothing detected

0: 1504x2688 1 car, 1 bus, 4 trucks, 1 bottle, 1 banana, 1 cake, 1 refrigerator, 25.2ms
Speed: 0.0ms preprocess, 25.2ms inference, 5.9ms postprocess per image at shape (1, 3, 1504, 2688)
nothing detected

0: 1504x26

In [11]:

import time

# Инициализация устройств и модели
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = YOLO('yolov8n.pt').to(device)
model.fuse()  # Оптимизация модели
model.half()  # Полуточная точность для GPU

# Оптимальные параметры
TARGET_SIZE = (640, 640)  # Оптимальный размер для YOLO
CONF_THRESH = 0.5  # Порог уверенности
IOU_THRESH = 0.4  # Порог для NMS
CLASSES = ['car', 'truck', 'bus', 'motorcycle']  # Интересующие классы

# Видеопоток
cap = cv2.VideoCapture("H:\stam_CV\cvtest.avi")
if not cap.isOpened():
    raise IOError("Cannot open video file")

# Зона детекции (полигон)
polygon = np.array([
    [0, 483], [1680, 460], [1680, 1400], [0, 1400]
], dtype=np.int32)

def preprocess(frame, device):
    """Подготовка кадра для модели"""
    frame_resized = cv2.resize(frame, TARGET_SIZE)
    frame_tensor = torch.from_numpy(frame_resized).to(device, non_blocking=True)
    frame_tensor = frame_tensor.half().permute(2, 0, 1).unsqueeze(0) / 255.0
    return frame_tensor

def scale_coords(boxes, frame_shape):
    """Масштабирование координат обратно к исходному размеру"""
    scale_x = frame_shape[1] / TARGET_SIZE[1]
    scale_y = frame_shape[0] / TARGET_SIZE[0]
    boxes[:, [0, 2]] *= scale_x
    boxes[:, [1, 3]] *= scale_y
    return boxes

def draw_results(frame, boxes, scores, class_ids):
    """Отрисовка результатов на кадре"""
    for box, score, class_id in zip(boxes, scores, class_ids):
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = f"{model.names[class_id]}: {score:.2f}"
        cv2.putText(frame, label, (x1, y1-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

# Основной цикл обработки
frame_count = 0
start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Подготовка кадра
    if device.type == 'cuda':
        frame_tensor = preprocess(frame, device)
        with torch.no_grad():
            results = model(frame_tensor)[0]
        boxes = results.boxes.xyxy.cpu().numpy()
        scores = results.boxes.conf.cpu().numpy()
        class_ids = results.boxes.cls.cpu().numpy().astype(int)
    else:
        results = model(frame)[0]
        boxes = results.boxes.xyxy.numpy()
        scores = results.boxes.conf.numpy()
        class_ids = results.boxes.cls.numpy().astype(int)
    
    # Фильтрация по классам и уверенности
    class_ids_to_keep = [class_id for class_id, class_name in model.names.items() if class_name in CLASSES]
    mask = (scores > CONF_THRESH) & np.isin(class_ids, class_ids_to_keep)
    boxes = boxes[mask]
    scores = scores[mask]
    class_ids = class_ids[mask]
    
    # Масштабирование координат
    if device.type == 'cuda':
        boxes = scale_coords(boxes, frame.shape)
    
    # Применение NMS
    indices = cv2.dnn.NMSBoxes(
        bboxes=boxes.tolist(),
        scores=scores.tolist(),
        score_threshold=CONF_THRESH,
        nms_threshold=IOU_THRESH
    )
    
    # Отрисовка результатов
    output_frame = frame.copy()
    if len(indices) > 0:
        boxes = boxes[indices]
        scores = scores[indices]
        class_ids = class_ids[indices]
        draw_results(output_frame, boxes, scores, class_ids)
    
    # Вывод FPS
    frame_count += 1
    fps = frame_count / (time.time() - start_time)
    cv2.putText(output_frame, f"FPS: {fps:.1f}", (10, 30),
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    cv2.imshow("Detection", output_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

<>:18: SyntaxWarning: invalid escape sequence '\s'
<>:18: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Alex\AppData\Local\Temp\ipykernel_5344\23922517.py:18: SyntaxWarning: invalid escape sequence '\s'
  cap = cv2.VideoCapture("H:\stam_CV\cvtest.avi")


Using device: cuda
YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

0: 640x640 1 person, 1 car, 1 truck, 9.5ms
Speed: 0.1ms preprocess, 9.5ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 2 cars, 1 truck, 10.8ms
Speed: 0.2ms preprocess, 10.8ms inference, 3.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 cars, 1 truck, 8.4ms
Speed: 0.1ms preprocess, 8.4ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 cars, 1 truck, 10.0ms
Speed: 0.1ms preprocess, 10.0ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 cars, 1 truck, 9.7ms
Speed: 0.1ms preprocess, 9.7ms inference, 3.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 cars, 1 truck, 9.0ms
Speed: 0.1ms preprocess, 9.0ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 cars, 1 truck, 9.3ms
Speed: 0.2ms preprocess, 9.3ms inference, 2.6ms postpro

In [ ]:

# Видео
cap = cv2.VideoCapture("H:\stam_CV\cvtest.avi")
# Заданный многоугольник
polygon = np.array([
    [0, 483], [1680, 460], [1680, 1400], [0, 1400]
], dtype=np.int32)

def box_intersects_polygon(box, polygon):
    """Проверка пересечения bbox с полигоном"""
    x1, y1, x2, y2 = map(int, box)
    box_points = [(x1, y1), (x2, y1), (x2, y2), (x1, y2)]
    return any(cv2.pointPolygonTest(polygon, point, False) >= 0 for point in box_points)

def draw_dashed_polygon(img, polygon, color=(0, 255, 0), thickness=2, gap=15):
    """Рисование пунктирного полигона"""
    for i in range(len(polygon)):
        pt1 = tuple(polygon[i])
        pt2 = tuple(polygon[(i + 1) % len(polygon)])
        cv2.line(img, pt1, pt2, color, thickness, lineType=cv2.LINE_AA)

def recognize_license_plate(frame, box):
    """Распознавание номера машины с помощью EasyOCR"""
    x1, y1, x2, y2 = map(int, box)
    plate_roi = frame[y1:y2, x1:x2]
    
    # Улучшаем контраст для лучшего распознавания
    gray = cv2.cvtColor(plate_roi, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Распознаем текст
    results = reader.readtext(thresh)
    if results:
        # Берем самый достоверный результат
        return results[0][1]
    return None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    output_frame = frame.copy()
    draw_dashed_polygon(output_frame, polygon)

    results = model(frame)[0]
    boxes = []
    scores = []
    license_plates = []

    for box in results.boxes.data.tolist():
        x1, y1, x2, y2, score, cls = box
        class_id = int(cls)
        class_name = model.names[class_id]

        if class_name in ['car', 'truck', 'motorcycle', 'bus']:
            if box_intersects_polygon((x1, y1, x2, y2), polygon):
                boxes.append([x1, y1, x2, y2])
                scores.append(score)
                # Распознаем номер
                plate_text = recognize_license_plate(frame, (x1, y1, x2, y2))
                license_plates.append(plate_text)

    # Применяем NMS
    indices = cv2.dnn.NMSBoxes(
        bboxes=[[int(x1), int(y1), int(x2 - x1), int(y2 - y1)] for x1, y1, x2, y2 in boxes],
        scores=scores,
        score_threshold=0.3,
        nms_threshold=0.4
    )

    vehicle_count = 0
    detected_plates = []
    
    for idx in indices:
        idx = idx[0] if isinstance(idx, (tuple, list, np.ndarray)) else idx
        x1, y1, x2, y2 = map(int, boxes[idx])
        vehicle_count += 1
        
        # Рисуем bounding box
        cv2.rectangle(output_frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
        
        # Добавляем распознанный номер (если есть)
        plate_text = license_plates[idx]
        if plate_text:
            cv2.putText(output_frame, plate_text, (x1, y1 - 10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            detected_plates.append(plate_text)

    # Формируем строку статуса
    if vehicle_count == 0:
        status_text = "Nothing detected"
    else:
        plates_text = ", ".join([p for p in detected_plates if p])
        status_text = f"{vehicle_count} vehicle(s) in zone: {plates_text}"
    
    # Выводим статус
    cv2.putText(output_frame, status_text, (30, 40), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.imshow("Detection", output_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Alex\AppData\Local\Temp\ipykernel_20912\443552764.py:2: SyntaxWarning: invalid escape sequence '\s'
  cap = cv2.VideoCapture("H:\stam_CV\cvtest.avi")


0: 384x640 2 persons, 2 cars, 1 truck, 86.5ms
Speed: 2.4ms preprocess, 86.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


h:\stam_CV\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\Alex\AppData\Local\Temp\ipykernel_20912\443552764.py:2: SyntaxWarning: invalid escape sequence '\s'
  cap = cv2.VideoCapture("H:\stam_CV\cvtest.avi")


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
